In [5]:
:ext DataKinds FlexibleContexts QuasiQuotes OverloadedStrings TemplateHaskell TypeApplications TypeOperators ViewPatterns

In [6]:
import DataGlue.Frames
import Frames.CSV

-- Since the used dataset as no header, let's define the column names.
datasource = "../datasets/iris.csv"
tableTypes' (rowGen datasource)
            { rowTypeName = "IrisRow"
            , columnNames = [ "Petal Length", "Petal Width", "Sepal Length" , "Sepal Width", "Iris Class" ]}

loadRows :: IO (Frame IrisRow)
loadRows = inCoreAoS (readTable datasource)

iris <- loadRows

In [7]:
-- Show dataframe structure.
describe iris

149x5 dataframe.
  column "Petal Length": Double
  column "Petal Width": Double
  column "Sepal Length": Double
  column "Sepal Width": Double
  column "Iris Class": Text

In [8]:
-- Show dataframe content.
iris

Petal Length,Petal Width,Sepal Length,Sepal Width,Iris Class
4.9,3.0,1.4,0.2,"""Iris-setosa"""
4.7,3.2,1.3,0.2,"""Iris-setosa"""
4.6,3.1,1.5,0.2,"""Iris-setosa"""
5.0,3.6,1.4,0.2,"""Iris-setosa"""
5.4,3.9,1.7,0.4,"""Iris-setosa"""
4.6,3.4,1.4,0.3,"""Iris-setosa"""
5.0,3.4,1.5,0.2,"""Iris-setosa"""
4.4,2.9,1.4,0.2,"""Iris-setosa"""
4.9,3.1,1.5,0.1,"""Iris-setosa"""
5.4,3.7,1.5,0.2,"""Iris-setosa"""


In [13]:
(train, test') = splitFrame 0 iris 0.8
test = dropColumn @IrisClass test'
length train
length test

119

30

In [27]:
splitTrainTestFrame seed df n col =
  let (train, pretest) = splitFrame seed df n
      test = dropColumn @IrisClass pretest
  in (train, test)

(tr, ts) = splitTrainTestFrame 0 iris 0.8 "_"

describe tr
describe ts
:t splitTrainTestFrame

119x5 dataframe.
  column "Petal Length": Double
  column "Petal Width": Double
  column "Sepal Length": Double
  column "Sepal Width": Double
  column "Iris Class": Text

30x4 dataframe.
  column "Petal Length": Double
  column "Petal Width": Double
  column "Sepal Length": Double
  column "Sepal Width": Double

splitTrainTestFrame :: forall (xs :: [*]) p1 p2.
(RealFrac p1, RecSubset Rec (Drop IrisClass xs) xs (RImage (Drop IrisClass xs) xs)) =>
Int -> Frame (Record xs) -> p1 -> p2 -> (Frame (Record xs), Frame (Record (Drop IrisClass xs)))